In [25]:
import seaborn as sns
import pandas as pd
from pathlib import Path
import sys
from logging import basicConfig, INFO
import ray
import pickle
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib as mpl

from simulation import *

%load_ext autoreload
%autoreload 2

mpl.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],  # Or Times, Computer Modern
})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Testing

In [26]:
alpha_prefs = 1
alpha_caps = 5
n_positions = 10
n_persons = 100
total_cap = 50
n_simulations = 3
sims = run_simulation(alpha_prefs=alpha_prefs, alpha_caps= alpha_caps, n_positions=n_positions, n_persons= n_persons, total_cap= total_cap, n_sims=n_simulations)


Running simulation with params: %s {'alpha_prefs': 1, 'alpha_caps': 5, 'n_positions': 10, 'n_persons': 100, 'total_cap': 50, 'n_sims': 3, 'rng': Generator(PCG64) at 0x30E708BA0}


Simulations:   0%|          | 0/108 [00:00<?, ?it/s]

In [27]:
df = make_df(sims)

# Evaluation

In [28]:
import os
import pandas as pd

subdir = Path('./sims_v3')
dataframes = [pd.read_csv(csv) for csv in subdir.glob("*.csv")]
df = pd.concat(dataframes, ignore_index=True)

# Visualisation

In [29]:
filters = [
    'n_roles',
    'n_persons',
    'gender_bias',
    'total_cap',
    'alpha_caps',
    'alpha_prefs'
]

def visualise(df, target):
    sns.lmplot(data=df, x="tvd", y=target, hue="quota", palette="Set2", height=4, col="quota", col_wrap=2, aspect=1)
    plt.show()

def filter_and_visualize(target, **kwargs):
    filtered_df = df.copy()

    for key, value in kwargs.items():
        filtered_df = filtered_df[filtered_df[key] == value]
    
    # Show visualization
    if not filtered_df.empty:
        visualise(filtered_df, target)
    else:
        print("No data matches the filter criteria.")

dropdowns = {col: widgets.Dropdown(options=sorted(df[col].unique()), description=col) for col in filters}
dropdowns['target'] = widgets.Dropdown(options=['total_util_perc', 'total_util', 'g0_util_perc'], description='target') 

# Link widgets to the output
ui = widgets.HBox(list(dropdowns.values()))
out = widgets.interactive_output(filter_and_visualize, dropdowns)

# Display widgets and output
display(ui, out)

Output()

In [30]:
filters = [
    'n_roles',
    'n_persons',
    'gender_bias',
    'total_cap',
    'alpha_caps',
    'alpha_prefs'
]
targets = ['total_util_perc', 'g0_util_perc', 'g1_util_perc']
#targets = ['total_util', 'g0_util', 'g1_util']

def visualise(df):
    df_melted = df.melt(id_vars=["tvd", "quota"], value_vars=targets, var_name="target", value_name="value")
    #df_melted = df_melted.dropna(subset=["tvd", "value"])
    df_melted[~(np.isfinite(df_melted["tvd"]) & np.isfinite(df_melted["value"]))] = 100 # high number of infinity. 
    
    # Plot with FacetGrid
    g = sns.FacetGrid(df_melted, col="quota", hue="target", palette="Set2", col_wrap=2, height=4, aspect=1)
    g.map(sns.regplot, "tvd", "value", scatter_kws={'alpha':0.5}, ci=None, robust=True)
    #g.map(sns.scatterplot, "tvd", "value", alpha=0.5)
    g.add_legend()
    spread = 1.1
    y_min = df_melted['value'].quantile(0.05)/spread
    y_max = df_melted['value'].quantile(0.95)*spread
    g.set(ylim=(y_min, y_max))
    plt.show()

def filter_and_visualize(**kwargs):
    filtered_df = df.copy()

    for key, value in kwargs.items():
        filtered_df = filtered_df[filtered_df[key] == value]
    
    # Show visualization
    if not filtered_df.empty:
        visualise(filtered_df)
    else:
        print("No data matches the filter criteria.")

dropdowns = {col: widgets.Dropdown(options=sorted(df[col].unique()), description=col) for col in filters}

# Link widgets to the output
ui = widgets.HBox(list(dropdowns.values()))
out = widgets.interactive_output(filter_and_visualize, dropdowns)

# Display widgets and output
display(ui, out)

Output()

## Plot

In [31]:
filters = [
    'n_roles',
    'n_persons',
    'total_cap',
]
targets = ['total_util_perc', 'g0_util_perc', 'g1_util_perc']
#targets = ['total_util', 'g0_util', 'g1_util']

def visualise(df, target):
    df_violin = df[['id', 'quota', 'gender_bias', target]]
    df_violin = df_violin.set_index(['id', 'quota'])
    df_violin = df_violin.pivot(columns='gender_bias', values=target)
    df_violin = df_violin[['NONE', 'LOW', 'MEDIUM', 'HIGH', 'VERY_HIGH', 'EXTREME']]
    sns.violinplot(data=df_violin, bw_adjust=.5, cut=1, linewidth=1, palette="Set3")
    plt.ylim(0.6, 1.1)
    plt.show()

def filter_and_visualize(**kwargs):
    target = kwargs.pop('target')
    tvd = kwargs.pop('tvd')
    tvd_dir = kwargs.pop('tvd_dir')
    filtered_df = df.copy()

    if tvd_dir == 'UP':
        filtered_df = filtered_df[filtered_df['tvd'] >= float(tvd)]
    else:
        filtered_df = filtered_df[filtered_df['tvd'] <= float(tvd)]

    for key, value in kwargs.items():
        filtered_df = filtered_df[filtered_df[key] == value]
    
    # Show visualization
    if not filtered_df.empty:
        visualise(filtered_df, target)
    else:
        print("No data matches the filter criteria.")

dropdowns = {col: widgets.Dropdown(options=sorted(df[col].unique()), description=col) for col in filters}
dropdowns['target'] = widgets.Dropdown(options=targets, description='target')
dropdowns['quota'] = widgets.Dropdown(options=['NONE', 'GTE20', 'GTE30', 'GTE40', 'EQU50'], description='quota')
dropdowns['tvd'] = widgets.Dropdown(options=['0.0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1.0'], description='tvd')
dropdowns['tvd_dir'] = widgets.Dropdown(options=['UP', 'DOWN'], description='tvd_dir')

# Link widgets to the output
ui = widgets.HBox(list(dropdowns.values()))
out = widgets.interactive_output(filter_and_visualize, dropdowns)

# Display widgets and output
display(ui, out)

Output()